In [ ]:
pip install nltk 

In [ ]:
pip install scikit-learn

In [1]:
#natural language tool kit
import pandas as pd
import numpy as np
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
netflix_df = pd.read_csv("netflixData.csv")
netflix_df.head()

,Show Id,Title,Description,Director,Genres,Cast,Production Country,Release Date,Rating,Duration,Imdb Score,Content Type,Date Added
0,cc1b6ed9-cf9e-4057-8303-34577fb54477,(Un)Well,This docuseries takes a deep dive into the luc...,NaN,Reality TV,NaN,United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
1,e2ef4e91-fb25-42ab-b485-be8e3b23dedb,#Alive,"As a grisly virus rampages a city, a lone man ...",Cho Il,"Horror Movies, International Movies, Thrillers","Yoo Ah-in, Park Shin-hye",South Korea,2020.0,TV-MA,99 min,6.2/10,Movie,"September 8, 2020"
2,b01b73b7-81f6-47a7-86d8-acb63080d525,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...","Sabina Fedeli, Anna Migotto","Documentaries, International Movies","Helen Mirren, Gengher Gatti",Italy,2019.0,TV-14,95 min,6.4/10,Movie,"July 1, 2020"
3,b6611af0-f53c-4a08-9ffa-9716dc57eb9c,#blackAF,Kenya Barris and his family navigate relations...,NaN,TV Comedies,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
4,7f2d4170-bab8-4d75-adc2-197f7124c070,#cats_the_mewvie,This pawesome documentary explores how our fel...,Michael Margolis,"Documentaries, International Movies",NaN,Canada,2020.0,TV-14,90 min,5.1/10,Movie,"February 5, 2020"


In [3]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5967 entries, 0 to 5966
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Show Id             5967 non-null   object 
 1   Title               5967 non-null   object 
 2   Description         5967 non-null   object 
 3   Director            3903 non-null   object 
 4   Genres              5967 non-null   object 
 5   Cast                5437 non-null   object 
 6   Production Country  5408 non-null   object 
 7   Release Date        5964 non-null   float64
 8   Rating              5963 non-null   object 
 9   Duration            5964 non-null   object 
 10  Imdb Score          5359 non-null   object 
 11  Content Type        5967 non-null   object 
 12  Date Added          4632 non-null   object 
dtypes: float64(1), object(12)
memory usage: 606.1+ KB


In [4]:
netflix_df.isnull()

,Show Id,Title,Description,Director,Genres,Cast,Production Country,Release Date,Rating,Duration,Imdb Score,Content Type,Date Added
0,False,False,False,True,False,True,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962,False,False,False,False,False,False,False,False,False,False,False,False,False
5963,False,False,False,True,False,False,True,False,False,False,True,False,True
5964,False,False,False,True,False,False,True,False,False,False,True,False,True
5965,False,False,False,False,False,False,False,False,False,False,False,False,False


In [5]:
netflix_df.isnull().sum()

Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64

In [6]:
required_nf_df = netflix_df[["Title","Description","Content Type", "Genres"]]
required_nf_df.head()

,Title,Description,Content Type,Genres
0,(Un)Well,This docuseries takes a deep dive into the luc...,TV Show,Reality TV
1,#Alive,"As a grisly virus rampages a city, a lone man ...",Movie,"Horror Movies, International Movies, Thrillers"
2,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...",Movie,"Documentaries, International Movies"
3,#blackAF,Kenya Barris and his family navigate relations...,TV Show,TV Comedies
4,#cats_the_mewvie,This pawesome documentary explores how our fel...,Movie,"Documentaries, International Movies"


In [7]:
#drop the rows which has null values
required_nf_df = required_nf_df.dropna()

In [8]:
required_nf_df.count()

Title           5967
Description     5967
Content Type    5967
Genres          5967
dtype: int64

In [9]:
#Clean the data
import nltk

In [13]:
nltk.download('stopwords') #stopwords: high frequency and low impact
stemmer = nltk.SnowballStemmer("english") #basically reducing large words into small root words is known as stemming
from nltk.corpus import stopwords
import string
stopword = set(stopwords.words('english'))

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [14]:
#Clean the data
import re
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text   


In [33]:
required_nf_df["Cleaned_Title"] = required_nf_df['Title'].apply(clean)
required_nf_df.head()

,Title,Description,Content Type,Genres,Cleaned_Title
0,unwel,This docuseries takes a deep dive into the luc...,TV Show,Reality TV,unwel
1,aliv,"As a grisly virus rampages a city, a lone man ...",Movie,"Horror Movies, International Movies, Thrillers",aliv
2,annefrank parallel stori,"Through her diary, Anne Frank's story is retol...",Movie,"Documentaries, International Movies",annefrank parallel stori
3,blackaf,Kenya Barris and his family navigate relations...,TV Show,TV Comedies,blackaf
4,catsthemewvi,This pawesome documentary explores how our fel...,Movie,"Documentaries, International Movies",catsthemewvi


3409          nikki glaser bangin
2097                    home game
865                        caliph
3860                      rangoon
4966                     last day
5955                          zoo
962     chhota bheem curs damyaan
2621                      lakshya
871                  camarón film
2148                         hugo
Name: Title, dtype: object

In [19]:
genreList = required_nf_df["Genres"].tolist()
tfidf = text.TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(genreList)
similarity = cosine_similarity(tfidf_matrix) 


In [34]:
indices = pd.Series(required_nf_df.index, index=required_nf_df["Cleaned_Title"]).drop_duplicates()

In [35]:
def netflix_recommendation(title, similarity = similarity):
    index = indices[title]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = similarity_scores[0:10]
    movieindices = [i[0] for i in similarity_scores]
    return required_nf_df['Title'].iloc[movieindices]

In [38]:
netflix_recommendation("rangoon")

0                        unwel
1                         aliv
2    annefrank  parallel stori
3                      blackaf
4                 catsthemewvi
5               friendbutmarri
6              friendbutmarri 
7                  realityhigh
8                             
9                        selfi
Name: Title, dtype: object